In [16]:
import numpy as np
import pandas as pd
import pylab
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib import cm

In [11]:
def load_data(X_filename, label_filename):
    print('load data.....')
    mnist_x = pd.read_csv(r"mnist_X.csv",header=None)
    mnist_y = pd.read_csv(r"mnist_label.csv",header=None)
    return mnist_x.values, mnist_y.values

def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

In [12]:
def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P



In [13]:
def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    # Implement PCA here
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y

In [14]:
def tsne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs t-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 1000
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 500
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.									# early exaggeration
    P = np.maximum(P, 1e-12)

    # Run iterations
    for iter in range(max_iter):

        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = -2. * np.dot(Y, Y.T)
        num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 10 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.

    # Return solution
    return Y,P,Q

In [17]:
if __name__ == "__main__":
    print("Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.")
    print("Running example on 5000 MNIST digits...")
    X, labels = load_data('mnist_X.csv', 'mnist_label.csv')
    labels = labels.flatten()
    #Y = tsne(X, 2, 50, 20.0)
    
    for per in range (3,6,1):
        perplexity = 2**per
        print ("perplexity ",perplexity)
        Y,P,Q = tsne(X, 2, 50, perplexity)
        
        data = [[[0 for i in range(2)] for j in range(0)] for k in range(5)]

        for i in range (5):
            idx = np.where(labels == i+1)
            data[i].append(Y[idx])
        plt.axis('equal')
        for i in range (5):
            print("i",i)
            plt.plot(np.squeeze(data[i])[:,0],np.squeeze(data[i])[:,1],marker='o',color=plt.cm.tab10(i+1),markersize=2,alpha =0.8,linestyle='none',label = str(i+1))
 
        plt.title("Symetric tNE with perplexity = " + str(perplexity))
        plt.legend(loc="best")
        plt.savefig("plots/tSNE"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()
        
        PFlatten = P.flatten()
        QFlatten = Q.flatten()
        plt.hist (PFlatten,bins=50,rwidth=1.0, log = True,density =True)
        plt.title("Histogram high-dimesion in log-scale")
        plt.savefig("plots/Hist_tSNE_P"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()
        plt.title("Histogram low-dimesion in log-scale")
        plt.hist (QFlatten,bins=50,rwidth=1.0, log = True,density =True)
        plt.savefig("plots/Hist_tSNE_Q"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()

Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.
Running example on 5000 MNIST digits...
load data.....
perplexity  8
Preprocessing the data using PCA...
Computing pairwise distances...
Computing P-values for point 0 of 5000...
Computing P-values for point 500 of 5000...
Computing P-values for point 1000 of 5000...
Computing P-values for point 1500 of 5000...
Computing P-values for point 2000 of 5000...
Computing P-values for point 2500 of 5000...
Computing P-values for point 3000 of 5000...
Computing P-values for point 3500 of 5000...
Computing P-values for point 4000 of 5000...
Computing P-values for point 4500 of 5000...
Mean value of sigma: 1.508087
Iteration 10: error is 29.727126
Iteration 20: error is 28.769413
Iteration 30: error is 24.914100
Iteration 40: error is 22.380368
Iteration 50: error is 21.323299
Iteration 60: error is 20.830491
Iteration 70: error is 20.503138
Iteration 80: error is 20.199442
Iteration 90: error is 19.987622
Iteration 100:

Computing P-values for point 2500 of 5000...
Computing P-values for point 3000 of 5000...
Computing P-values for point 3500 of 5000...
Computing P-values for point 4000 of 5000...
Computing P-values for point 4500 of 5000...
Mean value of sigma: 1.952543
Iteration 10: error is 24.891835
Iteration 20: error is 23.542762
Iteration 30: error is 19.666587
Iteration 40: error is 18.028597
Iteration 50: error is 17.467760
Iteration 60: error is 17.233150
Iteration 70: error is 17.094003
Iteration 80: error is 16.993266
Iteration 90: error is 16.928898
Iteration 100: error is 16.891008
Iteration 110: error is 2.501668
Iteration 120: error is 2.329584
Iteration 130: error is 2.198153
Iteration 140: error is 2.094187
Iteration 150: error is 2.007652
Iteration 160: error is 1.934763
Iteration 170: error is 1.872870
Iteration 180: error is 1.819742
Iteration 190: error is 1.773758
Iteration 200: error is 1.733526
Iteration 210: error is 1.698031
Iteration 220: error is 1.666475
Iteration 230: err